In [12]:
#importando as bibliotecas
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
try:
    nome_arquivo = 'C:/Users/maaf2/Downloads/COTAHIST_A2021/COTAHIST_A2021.csv'
    arquivo = open(nome_arquivo, 'r+')
except FileNotFoundError:
    arquivo = open(nome_arquivo, 'w+')
    
try:
    nome_arquivo = 'C:/Users/maaf2/Downloads/COTAHIST_A2021/previsao_siglas.csv'
    arquivo_previsao = open(nome_arquivo, 'r+')
except FileNotFoundError:
    arquivo_previsao = open(nome_arquivo, 'w+')
    
arquivo_previsao.writelines('Sigla;CODNEG;DATA DO PREGAO;real;previsao\n')

arq = open("C:/Users/maaf2/Downloads/COTAHIST_A2021/COTAHIST_A2021.TXT")
linhas = arq.readlines()
arquivo.writelines('DATA DO PREGAO;CODNEG;preco_abertura;preco_max;preco_minimo;preco_fechamento;qtd_negocios;volume_negocios\n')
for linha in linhas:
    if linha[0:2] == '01':
        linha_temp = ''
        linha_temp = linha[2:6]+'-'+linha[6:8]+'-'+linha[8:10]+';' #Data
        linha_temp = linha_temp+linha[12:24].strip()+';'           #Ação
        precoAbertura = int(linha[56:69])/100                            
        linha_temp = linha_temp+str(precoAbertura)+';'                    #Preço Abertura
        precoMaximo = int(linha[69:82])/100                            
        linha_temp = linha_temp+str(precoMaximo)+';'                    #Preço Maximo
        precoMinimo = int(linha[82:95])/100                            
        linha_temp = linha_temp+str(precoMinimo)+';'                    #Preço Minimo
        precoFechamento = int(linha[108:121])/100                            
        linha_temp = linha_temp+str(precoFechamento)+';'                    #Preço Fechamento
        linha_temp = linha_temp+linha[152:170]+';'           #Qtde de Negócio
        linha_temp = linha_temp+linha[170:188]+'\n'           #Volume de negócios
        arquivo.writelines(linha_temp)
        
arquivo.close()
arq.close()

In [14]:
df = pd.read_csv("C:\\Users\\maaf2\\Downloads\\COTAHIST_A2021\\COTAHIST_A2021.csv", delimiter=';')
df_ult_op = df[df['DATA DO PREGAO'] == '2021-08-10' ]
siglas = df_ult_op['CODNEG'].unique()

In [15]:
def recuperarDfSigla(sigla):
    return df[df['CODNEG'] == sigla ]

In [16]:
def recuperarQtdLinhasTreino(df_siglas):
    #verificando quantidade de linhas
    qtd_linhas = len(df_siglas)

    qtd_linhas_treino= round(.70 * qtd_linhas)
    return qtd_linhas_treino

In [17]:
def recuperarQtdLinhasTeste(df_siglas):
        #verificando quantidade de linhas
    qtd_linhas = len(df_siglas)

    qtd_linhas_treino= round(.70 * qtd_linhas)
    qtd_linhas_teste= qtd_linhas - qtd_linhas_treino 
    return qtd_linhas_teste

In [18]:
def recuperarQtdLinhasValidacao(df_siglas):
    qtd_linhas = len(df_siglas)
    qtd_linhas_validacao = qtd_linhas -1
    return qtd_linhas_validacao

In [19]:
def treinarRegressaoLinear(X_train_scale,y_train,X_test_scale):
    #treinamento usando regressão linear
    lr = linear_model.LinearRegression()
    lr.fit(X_train_scale, y_train)
    pred= lr.predict(X_test_scale)
    cd =r2_score(y_test, pred)

    print(f'Coeficiente de determinação:{cd * 100:.2f}')

In [20]:
def definirMelhoresFeatures(features, labels):
    #Escolhendo as melhores features com Kbest

    features_list = ('preco_abertura','preco_max','preco_minimo','qtd_negocios','volume_negocios','mm5d','mm21d')

    k_best_features = SelectKBest(k='all')
    k_best_features.fit_transform(features, labels)
    k_best_features_scores = k_best_features.scores_
    raw_pairs = zip(features_list[1:], k_best_features_scores)
    ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

    k_best_features_final = dict(ordered_pairs[:15])
    best_features = k_best_features_final.keys()
    return best_features

In [21]:
#d = {'sigla': [], 'data': [], 'real': [], 'previsao': [], 'valor': [], 'percentual': []}
df_teste = pd.DataFrame(columns=['sigla', 'data', 'real', 'previsao', 'valor', 'percentual'])
#df_teste = pd.DataFrame(data=d)
df_teste['data'] = pd.to_datetime(df_teste['data'], format='%Y-%m-%d')
df_teste["real"] = pd.to_numeric(df_teste["real"], downcast='float')
df_teste["previsao"] = pd.to_numeric(df_teste["previsao"], downcast='float')
df_teste["valor"] = pd.to_numeric(df_teste["valor"], downcast='float')
df_teste["percentual"] = pd.to_numeric(df_teste["percentual"], downcast='float')
df_teste.dtypes

s = 'ITUB4'
len(s)

5

In [22]:
contador = 0

df_resultado = pd.DataFrame(columns=['sigla', 'data', 'real', 'previsao', 'valor', 'percentual', 'coeficiencia'])
#df_resultado.set_index('sigla', inplace=True)

df_resultado['data'] = pd.to_datetime(df_resultado['data'], format='%Y-%m-%d')
df_resultado["real"] = pd.to_numeric(df_resultado["real"], downcast='float')
df_resultado["previsao"] = pd.to_numeric(df_resultado["previsao"], downcast='float')
df_resultado["valor"] = pd.to_numeric(df_resultado["valor"], downcast='float')
df_resultado["percentual"] = pd.to_numeric(df_resultado["percentual"], downcast='float')
df_resultado["coeficiencia"] = pd.to_numeric(df_resultado["coeficiencia"], downcast='float')

for sigla in siglas:
    try:
        if len(sigla) > 5:
            continue
        df_siglas = recuperarDfSigla(sigla)
        df_siglas['DATA DO PREGAO'] = pd.to_datetime(df_siglas['DATA DO PREGAO'], format='%Y-%m-%d')
        #criando novos campos de medias móveis
        df_siglas['mm5d'] = df_siglas['preco_fechamento'].rolling(5).mean()
        df_siglas['mm21d'] = df_siglas['preco_fechamento'].rolling(21).mean()
        #Empurrando para frente os valores das ações
        df_siglas['preco_fechamento'] = df_siglas['preco_fechamento'].shift(-1)
        #retirando os dados nulos
        df_siglas.dropna(inplace=True)
        #reindexando o data frame
        df_siglas = df_siglas.reset_index(drop=True)

        #separando as features e labels
        features = df_siglas.drop(['CODNEG', 'DATA DO PREGAO', 'preco_fechamento'], 1)
        labels = df_siglas['preco_fechamento']
        #separando as features escolhidas
        features = df_siglas.loc[:,definirMelhoresFeatures(features, labels)]
        scaler = MinMaxScaler().fit(features)
        features_scale = scaler.transform(features)
        #Separa os dados de treino teste e validação
        qtd_linhas_treino = recuperarQtdLinhasTreino(df_siglas)
        qtd_linhas_teste = recuperarQtdLinhasTeste(df_siglas)
        X_train = features[:qtd_linhas_treino]
        X_test = features[qtd_linhas_treino:qtd_linhas_treino + qtd_linhas_teste -1]

        y_train = labels[:qtd_linhas_treino]
        y_test = labels[qtd_linhas_treino:qtd_linhas_treino + qtd_linhas_teste -1]
        # Normalizando os dados de entrada(features)

        # Gerando o novo padrão
        scaler = MinMaxScaler()
        X_train_scale = scaler.fit_transform(X_train)  # Normalizando os dados de entrada(treinamento)
        X_test_scale  = scaler.transform(X_test)       # Normalizando os dados de entrada(teste)

        #treinamento usando regressão linear
        lr = linear_model.LinearRegression()
        lr.fit(X_train_scale, y_train)
        pred= lr.predict(X_test_scale)
        cd =r2_score(y_test, pred)

        #print(f'Coeficiente de determinação:{cd * 100:.2f}')

        valor_novo = features.tail(1)

        df_tmp = df[df['CODNEG'] == sigla ]

        #df_tmp['DATA DO PREGAO'] = pd.to_datetime(df_siglas['DATA DO PREGAO'], format='%Y-%m-%d')

        #executando a previsão

        previsao=scaler.transform(valor_novo)
        pred=lr.predict(previsao)
        #pred

        data_pregao_full=df_tmp['DATA DO PREGAO']
        data_pregao=data_pregao_full.tail(1)

        res_full=df_tmp['preco_fechamento']
        res=res_full.tail(1)

        #df_resultado=pd.DataFrame({'Sigla': sigla,'DATA DO PREGAO':data_pregao, 'real':res, 'previsao':pred,'valor':(pred-res),'percentual':(1-(res/pred))*100})
        df_resultado = df_resultado.append({'sigla': sigla, 'data':data_pregao.values[0],'real':"%.2f" %res.values[0],'previsao':"%.2f" %pred[0],'valor':"%.2f" %(pred[0]-res.values[0]),'percentual':"%.2f" %((1-(res.values[0]/pred[0]))*100),'coeficiencia':"%.2f" %(cd * 100)},ignore_index=True)
        #df_resultado = df_resultado.append({'sigla': sigla,'real':res.values[0]},ignore_index=True)

        contador = contador + 1

        #linha_sigla = ''
        #linha_sigla = sigla+';'+data_pregao+';'+str(res)+';'+str(pred)+'\n'
        #arquivo_previsao.writelines(linha_sigla)

        #print(df_tmp)
        print(contador)
        #if contador == 10:
        #    break
    except ValueError:
       print("Erro na sigla: "+sigla)

df_resultado["percentual"] = pd.to_numeric(df_resultado["percentual"], downcast='float')
df_resultado = df_resultado.sort_values(by=['percentual'], ascending=False)
df_resultado.to_csv("C:/Users/maaf2/Downloads/COTAHIST_A2021/previsao_siglas.csv", index = True, sep=";")
#arquivo_previsao.close()

C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['DATA DO PREGAO'] = pd.to_datetime(df_siglas['DATA DO PREGAO'], format='%Y-%m-%d')
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['mm5d'] = df_siglas['preco_fechamento'].rolling(5).mean()
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:21: SettingWithCopyWarning: 
A value is trying to be set on

1
2


C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['DATA DO PREGAO'] = pd.to_datetime(df_siglas['DATA DO PREGAO'], format='%Y-%m-%d')
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['mm5d'] = df_siglas['preco_fechamento'].rolling(5).mean()
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:21: SettingWithCopyWarning: 
A value is trying to be set on

3
4


C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['DATA DO PREGAO'] = pd.to_datetime(df_siglas['DATA DO PREGAO'], format='%Y-%m-%d')
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['mm5d'] = df_siglas['preco_fechamento'].rolling(5).mean()
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:21: SettingWithCopyWarning: 
A value is trying to be set on

...

C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['DATA DO PREGAO'] = pd.to_datetime(df_siglas['DATA DO PREGAO'], format='%Y-%m-%d')
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['mm5d'] = df_siglas['preco_fechamento'].rolling(5).mean()
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:21: SettingWithCopyWarning: 
A value is trying to be set on

372
373


C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['DATA DO PREGAO'] = pd.to_datetime(df_siglas['DATA DO PREGAO'], format='%Y-%m-%d')
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['mm5d'] = df_siglas['preco_fechamento'].rolling(5).mean()
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:21: SettingWithCopyWarning: 
A value is trying to be set on

374
Erro na sigla: CVCB9
Erro na sigla: GFSA1


C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['DATA DO PREGAO'] = pd.to_datetime(df_siglas['DATA DO PREGAO'], format='%Y-%m-%d')
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_siglas['mm5d'] = df_siglas['preco_fechamento'].rolling(5).mean()
C:\Users\maaf2\AppData\Local\Temp/ipykernel_12404/3064955212.py:21: SettingWithCopyWarning: 
A value is trying to be set on